In [ ]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

### Task 1: Data Preparation

In this task we'll be collecting, and then parsing, our data.

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

In [ ]:
movie_data = CSVLoader(file_path="/content/test_data.csv")

data = movie_data.load()

In [ ]:
len(data)

16

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100, # the character length of the chunk
    chunk_overlap = 50, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

In [ ]:
movie_documents = text_splitter.transform_documents(data)

In [ ]:
len(movie_documents)

90

In [ ]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 117.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vector_store = FAISS.from_documents(movie_documents, embedder)

In [ ]:
query = "List some movies that are not good at all"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

﻿ID: 19773
Title: Dusk of Humanity
Review: Worse
Genre: Drama
special effects but lacks substantial character development.
﻿ID: 10021
Title: Future's Past
Review: Not so good
Genre: Sci-fi
﻿ID: 10234
Title: Echoes of Silence
Review: Not bad
Genre: Drama


In [ ]:
query = "List some movies that are the best to watch"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 3)

for page in docs:
  print(page.page_content)

﻿ID: 10837
Title: The Garden of Time
Review: Good
Genre: Drama
﻿ID: 19773
Title: Dusk of Humanity
Review: Worse
Genre: Drama
to restore balance. The film blends action, magic, and diverse landscapes but struggles with pacing


In [ ]:
query = "List some movies that not too bad to watch"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 3)

for page in docs:
  print(page.page_content)

﻿ID: 19773
Title: Dusk of Humanity
Review: Worse
Genre: Drama
﻿ID: 10837
Title: The Garden of Time
Review: Good
Genre: Drama
﻿ID: 10001
Title: Sherlock Holmes
Review: Not bad
Genre: Crime


In [ ]:
query = "What are the roles of Robert and Watson in the movie Sherlock Holmes?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 3)

# for page in docs:
#   print(page.page_content)

In [ ]:
docs

[Document(page_content='Summary: Sherlock Holmes is a crime thriller movie starring Robert Downy Jr and Jason Watson. Jason', metadata={'source': '/content/test_data.csv', 'row': 0}),
 Document(page_content='of a Detective. This movie is based on various crimes and how Watson and Robert solves these crime', metadata={'source': '/content/test_data.csv', 'row': 0}),
 Document(page_content='and how Watson and Robert solves these crime mystries.', metadata={'source': '/content/test_data.csv', 'row': 0})]

In [ ]:
%%timeit -n 1 -r 1
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

9.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

8.16 ms ± 33.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
!pip install huggingface-hub -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Now we need to pack it into a `pipeline` for compatability with `langchain`!

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.5,
    max_new_tokens=256
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

Now we can set up our chain.

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Now that it's set-up, let's test it out!

In [ ]:
qa_with_sources_chain({"query" : "What is the role of Robert and Jason in this movie?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What is the role of Robert and Jason in this movie?',
 'result': ' Robert Downy Jr plays the role of Sherlock Holmes, a detective, while Jason Watson plays the role of Dr. John Watson.',
 'source_documents': [Document(page_content='starring Robert Downy Jr and Jason Watson. Jason is a doctor in the movie while Robert plays a role', metadata={'source': '/content/test_data.csv', 'row': 0}),
  Document(page_content='a doctor in the movie while Robert plays a role of a Detective. This movie is based on various', metadata={'source': '/content/test_data.csv', 'row': 0}),
  Document(page_content='Summary: Sherlock Holmes is a crime thriller movie starring Robert Downy Jr and Jason Watson. Jason', metadata={'source': '/content/test_data.csv', 'row': 0}),
  Document(page_content='of a Detective. This movie is based on various crimes and how Watson and Robert solves these crime', metadata={'source': '/content/test_data.csv', 'row': 0})]}

In [ ]:
qa_with_sources_chain({"query" : "List the movies which are not bad at all"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'List the movies which are not bad at all',
 'result': ' Based on the reviews, the only movie that is not bad is Sherlock Holmes.',
 'source_documents': [Document(page_content='\ufeffID: 19773\nTitle: Dusk of Humanity\nReview: Worse\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 10}),
  Document(page_content='special effects but lacks substantial character development.', metadata={'source': '/content/test_data.csv', 'row': 7}),
  Document(page_content='\ufeffID: 10837\nTitle: Neon Shadows\nReview: Bad\nGenre: Action', metadata={'source': '/content/test_data.csv', 'row': 5}),
  Document(page_content='\ufeffID: 10001\nTitle: Sherlock Holmes\nReview: Not bad\nGenre: Crime', metadata={'source': '/content/test_data.csv', 'row': 0})]}

In [ ]:
qa_with_sources_chain({"query" : "List the movies which are not too bad to watch"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'List the movies which are not too bad to watch',
 'result': ' Based on the reviews, the movies that are not too bad to watch are "The Garden of Time" and "Sherlock Holmes".',
 'source_documents': [Document(page_content='\ufeffID: 19773\nTitle: Dusk of Humanity\nReview: Worse\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 10}),
  Document(page_content='\ufeffID: 10837\nTitle: The Garden of Time\nReview: Good\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 6}),
  Document(page_content='\ufeffID: 10001\nTitle: Sherlock Holmes\nReview: Not bad\nGenre: Crime', metadata={'source': '/content/test_data.csv', 'row': 0}),
  Document(page_content='special effects but lacks substantial character development.', metadata={'source': '/content/test_data.csv', 'row': 7})]}

In [ ]:
qa_with_sources_chain({"query" : "List the movies which are good to watch"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'List the movies which are good to watch',
 'result': ' Based on the reviews, the movies that are good to watch are "Sherlock Holmes" and "The Garden of Time".',
 'source_documents': [Document(page_content='\ufeffID: 10837\nTitle: The Garden of Time\nReview: Good\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 6}),
  Document(page_content='\ufeffID: 19773\nTitle: Dusk of Humanity\nReview: Worse\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 10}),
  Document(page_content='\ufeffID: 10001\nTitle: Sherlock Holmes\nReview: Not bad\nGenre: Crime', metadata={'source': '/content/test_data.csv', 'row': 0}),
  Document(page_content='\ufeffID: 17638\nTitle: Ashes of Tomorrow\nReview: Not good\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 13})]}

In [ ]:
qa_with_sources_chain({"query" : "List the movies which are best to watch"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'List the movies which are best to watch',
 'result': '\n\nBased on the reviews, the best movie to watch would be "The Garden of Time" with a rating of "Good". The other two movies, "Dusk of Humanity" and "Ashes of Tomorrow" have negative reviews with ratings of "Worse" and "Not good" respectively. Therefore, it is not recommended to watch these two movies.',
 'source_documents': [Document(page_content='\ufeffID: 10837\nTitle: The Garden of Time\nReview: Good\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 6}),
  Document(page_content='\ufeffID: 19773\nTitle: Dusk of Humanity\nReview: Worse\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 10}),
  Document(page_content='to restore balance. The film blends action, magic, and diverse landscapes but struggles with pacing', metadata={'source': '/content/test_data.csv', 'row': 9}),
  Document(page_content='\ufeffID: 17638\nTitle: Ashes of Tomorrow\nReview: Not good\nGenre: Drama', metadata={

In [ ]:
qa_with_sources_chain({"query" : "List the movies which are worse to watch"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'List the movies which are worse to watch',
 'result': ' Based on the reviews, the movies that are worse to watch are "Dusk of Humanity" with a review of "Worse" and "The Garden of Time" with a review of "Good but lacks substance."',
 'source_documents': [Document(page_content='\ufeffID: 19773\nTitle: Dusk of Humanity\nReview: Worse\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 10}),
  Document(page_content='future. The narrative twists and ethical dilemmas keep viewers on edge', metadata={'source': '/content/test_data.csv', 'row': 2}),
  Document(page_content='\ufeffID: 10837\nTitle: The Garden of Time\nReview: Good\nGenre: Drama', metadata={'source': '/content/test_data.csv', 'row': 6}),
  Document(page_content='tornado in history. It’s a thrilling ride with cutting-edge special effects but lacks substantial', metadata={'source': '/content/test_data.csv', 'row': 7})]}